## <center>tensorboard<center>
    
在训练过程中，主要用到了tf.summary()的各类方法，能够保存训练过程以及参数分布图并在tensorboard显示。
histograms:变量显示
tf.summary.histogram

scalars:标量显示
tf.summary.scalar


text:展示文本输入类型为Tensor
tf.summary.text

tf.summary.merge(inputs, collections=None, name=None)
合并summaries

该op创建了一个summary协议缓冲区，它包含了输入的summaries的所有value的union


http://www.cnblogs.com/fydeblog/p/7429344.html
tensorboard --logdir logs


In [1]:
import tensorflow as tf
import numpy as np

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
## prepare the original data
with tf.name_scope('data'):
    x_data = np.random.rand(100).astype(np.float32)
    y_data = 0.3 * x_data + 0.1

##creat parameters
with tf.name_scope('parameters'):
    with tf.name_scope('weights'):
        weight = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
        tf.summary.histogram('weight', weight)
    with tf.name_scope('biases'):
        bias = tf.Variable(tf.zeros([1]))
        tf.summary.histogram('bias', bias)
        
##get y_prediction
with tf.name_scope('y_prediction'):
    y_prediction = weight * x_data + bias
    
##compute the loss
with tf.name_scope('loss'):
    #  均方误差mean squareerror
    loss = tf.reduce_mean(tf.square(y_data - y_prediction))  # square对a里的每一个元素求平方
    tf.summary.scalar('loss', loss)
    
##creat optimizer
optimizer = tf.train.GradientDescentOptimizer(0.5)
# creat train ,minimize the loss
with tf.name_scope('train'):
    train = optimizer.minimize(loss)
# creat init
with tf.name_scope('init'):
    init = tf.global_variables_initializer()

In [3]:
##creat a Session
sess = tf.Session()
# merged
merged = tf.summary.merge_all()
##initialize
writer = tf.summary.FileWriter("logs/", sess.graph)
sess.run(init)
## Loop
for step in range(101):
    sess.run(train)
    rs = sess.run(merged)
    writer.add_summary(rs, step)

### 计算图可视化

- tf.train.write_graph
Tensorflow也可以存储计算图，调用tf.train.write_graph（）方法可以把图定义存储下来。
也可以在TesnsorBoard中展示该图。

- tf.placeholder(dtype, shape=None, name=None)
此函数可以理解为形参，用于定义过程，在执行的时候再赋具体的值

In [ ]:
#设置两个乘数，用占位符表示
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
#设置乘积
output1 = tf.multiply(input1, input2)
output2 = input1 - input2
with tf.Session() as sess:
  #用feed_dict以字典的方式填充占位
    print(sess.run([output1,output2], feed_dict={input1:[8.],input2:[2.]}))
    
#加上这行代码，就可以可视化。指定文件的路径，没有会自动生成。指定数据源：计算图
#将计算图上面的数据写入events文件里面，tensorboard可以同时从events文件里面读取数据并显示在web端
writer=tf.summary.FileWriter('./first_graph',sess.graph)
writer.flush()
writer.close()
sess.close()